In [11]:
#get credentials

import os
from dotenv import load_dotenv
from notion_client import Client

load_dotenv()

NOTION_API_TOKEN = os.getenv('NOTION_SENSENETS_TOKEN')
URLS_DATABASE_ID = os.getenv('NOTION_URLS_DATABASE_ID')
POSTS_DATABASE_ID = os.getenv('NOTION_POSTS_DATABASE_ID')

HEADERS = {
    'Authorization': f'Bearer {NOTION_API_TOKEN}',
    'Content-Type': 'application/json',
    'Notion-Version': '2022-06-28'
}
notion = Client(auth=NOTION_API_TOKEN)


In [47]:
import json


# Function to search for a URL in the URLs database
def search_url_in_database(url):
    print('in search url')
    response = notion.databases.query(
        **{
            "database_id": URLS_DATABASE_ID,
            "filter": {
                "property": "URL",
                "url": {
                    "equals": url
                }
            }
        }
    )
    
    if response["results"]:
        print(f"URL found in database: {url}")
        return response["results"][0]["id"]  # Return the first matched page ID if the URL exists
    else:
        print(f"URL not found, will create a new page: {url}")
        return None

# Function to create a new page in the URLs database
def create_url_page(url):
    print("in create page")
    response = notion.pages.create(
        **{
            "parent": {"database_id": URLS_DATABASE_ID},
            "properties": {
                "Name": {
                    "title": [
                        {
                            "text": {"content": url}
                        }
                    ]
                },
                "URL": {
                    "url": url
                }
            }
        }
    )
    
    print(f"Created new page for URL: {url}")
    return response["id"]  # Return the page ID of the created URL page

# Function to search for a post in the Posts database by title
def search_post_in_database(post_title):
    response = notion.databases.query(
        **{
            "database_id": POSTS_DATABASE_ID,
            "filter": {
                "property": "Name",
                "title": {
                    "equals": post_title
                }
            }
        }
    )
    
    if response["results"]:
        print(f"Post '{post_title}' found in the database.")
        return response["results"][0]["id"]  # Return the first matched post ID if it exists
    else:
        print(f"Post '{post_title}' not found, will create a new page.")
        return None

# Function to create a new page in the Posts database
def create_post_page(post_title, recommendation_ids,post_text,post_creators_account):
    response = notion.pages.create(
        **{
            "parent": {"database_id": POSTS_DATABASE_ID},
            "properties": {
                "Name": {
                    "title": [
                        {
                            "text": {"content": post_title}
                        }
                    ]
                },
                "Recommends": {
                    "relation": [{"id": rec_id} for rec_id in recommendation_ids]
                },
                "Text": {
                    "rich_text": [
                        {
                            "text": {"content": post_text}
                        }
                    ]
                },
                "Creators Handle": {
                    "rich_text": [
                        {
                            "text": {"content": post_creators_account}
                        }
                    ]
                }
            }
        }
    )
    print(f"Created new post page for '{post_title}'.")
    return response["id"]  # Return the page ID of the created post page

# Function to update the recommendations field in the Posts database
def update_post_recommendations(post_id, recommendation_ids,post_text,post_creators_account):
    notion.pages.update(
        **{
            "page_id": post_id,
            "properties": {
                "Recommends": {
                    "relation": [{"id": page_id} for page_id in recommendation_ids]
                },
                "Text": {
                    "rich_text": [
                        {
                            "text": {"content": post_text}
                        }
                    ]
                },
                "Creators Handle": {
                    "rich_text": [
                        {
                            "text": {"content": post_creators_account}
                        }
                    ]
                }
            }
        }
    )
    print(f"Updated post {post_id} with new recommendations.")

# Step 3: Process the JSON file
def process_json(json_file):
    with open(json_file) as f:
        data = json.load(f)

        for post in data:
            post_name = create_post_name(post)  # Assuming the data has a Title field to identify posts
            recommendations = post.get('Recommendations', [])
            # First, search if the Post already exists
            post_id = search_post_in_database(post_name)
            post_text = post['Content']
            post_creators_account = post["User"]
            
            print(f"Processing Post ID: {post_id}")
            if recommendations:
                recommendation_ids = []
                
                for url in recommendations:
                    url = url.strip()
                    
                    # First, search if the URL already exists
                    url_page_id = search_url_in_database(url)
                    
                    if not url_page_id:  # If not found, create a new page for the URL
                        url_page_id = create_url_page(url)

                    
                    if url_page_id:
                        recommendation_ids.append(url_page_id)
                
            # If the post exists, update it, otherwise create a new post
            if post_id:
                update_post_recommendations(post_id, recommendation_ids,post_text,post_creators_account)
            else:
                create_post_page(post_name, recommendation_ids,post_text,post_creators_account)


#Create post name
def create_post_name(post):
    user_name = post['User']  # Get the user's name
    content = post['Content']  # Get the post content
    short_content = content[:60]  # Extract the first 10 characters of the content
    
    # Create the post name as "User name : First 10 characters of content"
    return f"{user_name}: {short_content}"

In [48]:
# Call the function to process the JSON file
process_json('data/test_json.json')

Post 'John Doe: Artificial intelligence is revolutionizing industries with i' not found, will create a new page.
Processing Post ID: None
in search url
URL found in database: https://example.com/ai1
in search url
URL found in database: https://example.com/ai2
Created new post page for 'John Doe: Artificial intelligence is revolutionizing industries with i'.
Post 'Jane Smith: Data science helps uncover insights from vast amounts of dat' not found, will create a new page.
Processing Post ID: None
in search url
URL found in database: https://example.com/ds1
Created new post page for 'Jane Smith: Data science helps uncover insights from vast amounts of dat'.
Post 'Alice Johnson: Machine learning algorithms are key to developing intelligen' not found, will create a new page.
Processing Post ID: None
in search url
URL found in database: https://example.com/ml1
in search url
URL found in database: https://example.com/ml2
Created new post page for 'Alice Johnson: Machine learning algorithms ar

In [23]:
#Testing access to Notion databases 

# Function to read all entries from the Posts database
def read_posts_database():
    try:
        response = notion.databases.query(database_id=POSTS_DATABASE_ID)
        results = response.get("results", [])
        print(f"Posts Database contains {len(results)} entries.")
        for entry in results:
            print(f"Post ID: {entry['id']}, Title: {entry['properties']['Name']['title'][0]['text']['content']}")
    except Exception as e:
        print(f"Error reading Posts Database: {e}")

# Function to read all entries from the URLs database
def read_urls_database():
    try:
        response = notion.databases.query(database_id=URLS_DATABASE_ID)
        results = response.get("results", [])
        print(f"URLs Database contains {len(results)} entries.")
        for entry in results:
            print(f"URL Page ID: {entry['id']}, URL: {entry['properties']['URL']['url']}")
    except Exception as e:
        print(f"Error reading URLs Database: {e}")

# Test reading the databases
print("Reading Posts Database:")
read_posts_database()

print("\nReading URLs Database:")
read_urls_database()


Reading Posts Database:
Posts Database contains 3 entries.
Post ID: 26155953-b777-45d1-9613-320d671a539c, Title: Hihi
Post ID: 5fa16e7e-93e8-4955-af0e-6e42c68b6881, Title: Lala
Post ID: bcfc1994-ad55-41f2-8c30-ac8baabef624, Title: Shalom

Reading URLs Database:
URLs Database contains 1 entries.
URL Page ID: 4d0830cf-0b5a-439f-bf72-2e6a0a888dee, URL: www.example.org
